In [1]:
import pandas as pd
from web3 import Web3
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()

In [2]:
miner_txs = pd.read_csv("../data/tcash_miner_transactions.csv")

In [3]:
miner_txs.head()

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price
0,0x3a22fe99fd6c27b507760cb20a1d3ffae9ccd320dca8...,1122,142,0x80e179c4a2f057be91573b32bece53888fb48311,0x746aebc06d2ae31b71ac51429a19d54e797878e9,0,434790,232500000000,0x7d9af914000000000000000000000000000000000000...,8009441,...,NaN,NaN,1,2021-04-03 19:23:24 UTC,12168479,0xb88d592cf1a1d399e2621b8ca7aef638d9d7b3dad39b...,NaN,NaN,NaN,232500000000
1,0x3331a615c119a9958b4429e0789674d5219aa16f23d5...,1678,168,0xbe4d1e137a24af091be80ae58d652279665e3a27,0x746aebc06d2ae31b71ac51429a19d54e797878e9,0,434790,196500000000,0x7d9af914000000000000000000000000000000000000...,12080586,...,NaN,NaN,1,2021-03-31 23:25:33 UTC,12150112,0xb97682f9cb2da20ec91feb61eb44fee72a16d7924e19...,NaN,NaN,NaN,196500000000
2,0x3f23f2f61211ae1851fcadcecab419dff50027988eaa...,241,170,0x146ccb0e64cc3f80e09afcf52005b40dbd5f286d,0x746aebc06d2ae31b71ac51429a19d54e797878e9,0,506880,110100000000,0x80bc50cc000000000000000000000000000000000000...,9073076,...,NaN,NaN,1,2021-04-08 04:29:03 UTC,12197020,0xbc002fe784105836c2ebff4a3c941871061ac576db54...,NaN,NaN,NaN,110100000000
3,0x5097c46458696ec07fecf989c60cfb26ec37eabc3734...,6310,83,0xbe4d1e137a24af091be80ae58d652279665e3a27,0x746aebc06d2ae31b71ac51429a19d54e797878e9,0,400000,17500000000,0x7d9af914000000000000000000000000000000000000...,4131879,...,NaN,NaN,1,2021-07-11 08:40:51 UTC,12804973,0x70008e988a388c9b1b691540c35800584eb5b0bcb7c9...,NaN,NaN,NaN,17500000000
4,0x78b487413b9996ff504bf24b7ee4109f4e74acd3f66d...,7023,137,0xbe4d1e137a24af091be80ae58d652279665e3a27,0x746aebc06d2ae31b71ac51429a19d54e797878e9,0,400000,51000000730,0x7d9af914000000000000000000000000000000000000...,9509885,...,NaN,NaN,1,2021-08-03 15:34:58 UTC,12953119,0x59aba14dac8f1f4f09e14a7a70b4e4e8e9686a487194...,NaN,NaN,NaN,51000000730


In [4]:
'''
Set true and false variables to evaluate contract abi as a python list correctly.
'''

true = True
false = False
miner_address = "0x746aebc06d2ae31b71ac51429a19d54e797878e9"
miner_abi = '[{"inputs":[{"internalType":"bytes32","name":"_rewardSwap","type":"bytes32"},{"internalType":"bytes32","name":"_governance","type":"bytes32"},{"internalType":"bytes32","name":"_tornadoTrees","type":"bytes32"},{"internalType":"bytes32[3]","name":"_verifiers","type":"bytes32[3]"},{"internalType":"bytes32","name":"_accountRoot","type":"bytes32"},{"components":[{"internalType":"bytes32","name":"instance","type":"bytes32"},{"internalType":"uint256","name":"value","type":"uint256"}],"internalType":"struct Miner.Rate[]","name":"_rates","type":"tuple[]"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes32","name":"commitment","type":"bytes32"},{"indexed":false,"internalType":"bytes32","name":"nullifier","type":"bytes32"},{"indexed":false,"internalType":"bytes","name":"encryptedAccount","type":"bytes"},{"indexed":false,"internalType":"uint256","name":"index","type":"uint256"}],"name":"NewAccount","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"instance","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"RateChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"reward","type":"address"},{"indexed":false,"internalType":"address","name":"withdraw","type":"address"},{"indexed":false,"internalType":"address","name":"treeUpdate","type":"address"}],"name":"VerifiersUpdated","type":"event"},{"inputs":[],"name":"ACCOUNT_ROOT_HISTORY_SIZE","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"accountCount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"name":"accountNullifiers","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"accountRoots","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes[]","name":"_rewardArgs","type":"bytes[]"}],"name":"batchReward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32[]","name":"domains","type":"bytes32[]"}],"name":"bulkResolve","outputs":[{"internalType":"address[]","name":"result","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getLastAccountRoot","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"governance","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_root","type":"bytes32"},{"internalType":"uint256","name":"_index","type":"uint256"}],"name":"isKnownAccountRoot","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"rates","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"node","type":"bytes32"}],"name":"resolve","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes","name":"_proof","type":"bytes"},{"components":[{"internalType":"uint256","name":"rate","type":"uint256"},{"internalType":"uint256","name":"fee","type":"uint256"},{"internalType":"address","name":"instance","type":"address"},{"internalType":"bytes32","name":"rewardNullifier","type":"bytes32"},{"internalType":"bytes32","name":"extDataHash","type":"bytes32"},{"internalType":"bytes32","name":"depositRoot","type":"bytes32"},{"internalType":"bytes32","name":"withdrawalRoot","type":"bytes32"},{"components":[{"internalType":"address","name":"relayer","type":"address"},{"internalType":"bytes","name":"encryptedAccount","type":"bytes"}],"internalType":"struct Miner.RewardExtData","name":"extData","type":"tuple"},{"components":[{"internalType":"bytes32","name":"inputRoot","type":"bytes32"},{"internalType":"bytes32","name":"inputNullifierHash","type":"bytes32"},{"internalType":"bytes32","name":"outputRoot","type":"bytes32"},{"internalType":"uint256","name":"outputPathIndices","type":"uint256"},{"internalType":"bytes32","name":"outputCommitment","type":"bytes32"}],"internalType":"struct Miner.AccountUpdate","name":"account","type":"tuple"}],"internalType":"struct Miner.RewardArgs","name":"_args","type":"tuple"},{"internalType":"bytes","name":"_treeUpdateProof","type":"bytes"},{"components":[{"internalType":"bytes32","name":"oldRoot","type":"bytes32"},{"internalType":"bytes32","name":"newRoot","type":"bytes32"},{"internalType":"bytes32","name":"leaf","type":"bytes32"},{"internalType":"uint256","name":"pathIndices","type":"uint256"}],"internalType":"struct Miner.TreeUpdateArgs","name":"_treeUpdateArgs","type":"tuple"}],"name":"reward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes","name":"_proof","type":"bytes"},{"components":[{"internalType":"uint256","name":"rate","type":"uint256"},{"internalType":"uint256","name":"fee","type":"uint256"},{"internalType":"address","name":"instance","type":"address"},{"internalType":"bytes32","name":"rewardNullifier","type":"bytes32"},{"internalType":"bytes32","name":"extDataHash","type":"bytes32"},{"internalType":"bytes32","name":"depositRoot","type":"bytes32"},{"internalType":"bytes32","name":"withdrawalRoot","type":"bytes32"},{"components":[{"internalType":"address","name":"relayer","type":"address"},{"internalType":"bytes","name":"encryptedAccount","type":"bytes"}],"internalType":"struct Miner.RewardExtData","name":"extData","type":"tuple"},{"components":[{"internalType":"bytes32","name":"inputRoot","type":"bytes32"},{"internalType":"bytes32","name":"inputNullifierHash","type":"bytes32"},{"internalType":"bytes32","name":"outputRoot","type":"bytes32"},{"internalType":"uint256","name":"outputPathIndices","type":"uint256"},{"internalType":"bytes32","name":"outputCommitment","type":"bytes32"}],"internalType":"struct Miner.AccountUpdate","name":"account","type":"tuple"}],"internalType":"struct Miner.RewardArgs","name":"_args","type":"tuple"}],"name":"reward","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"name":"rewardNullifiers","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"rewardSwap","outputs":[{"internalType":"contract IRewardSwap","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"rewardVerifier","outputs":[{"internalType":"contract IVerifier","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_newWeight","type":"uint256"}],"name":"setPoolWeight","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"components":[{"internalType":"bytes32","name":"instance","type":"bytes32"},{"internalType":"uint256","name":"value","type":"uint256"}],"internalType":"struct Miner.Rate[]","name":"_rates","type":"tuple[]"}],"name":"setRates","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract TornadoTrees","name":"_tornadoTrees","type":"address"}],"name":"setTornadoTreesContract","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract IVerifier[3]","name":"_verifiers","type":"address[3]"}],"name":"setVerifiers","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"tornadoTrees","outputs":[{"internalType":"contract TornadoTrees","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"treeUpdateVerifier","outputs":[{"internalType":"contract IVerifier","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes","name":"_proof","type":"bytes"},{"components":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"bytes32","name":"extDataHash","type":"bytes32"},{"components":[{"internalType":"uint256","name":"fee","type":"uint256"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"address","name":"relayer","type":"address"},{"internalType":"bytes","name":"encryptedAccount","type":"bytes"}],"internalType":"struct Miner.WithdrawExtData","name":"extData","type":"tuple"},{"components":[{"internalType":"bytes32","name":"inputRoot","type":"bytes32"},{"internalType":"bytes32","name":"inputNullifierHash","type":"bytes32"},{"internalType":"bytes32","name":"outputRoot","type":"bytes32"},{"internalType":"uint256","name":"outputPathIndices","type":"uint256"},{"internalType":"bytes32","name":"outputCommitment","type":"bytes32"}],"internalType":"struct Miner.AccountUpdate","name":"account","type":"tuple"}],"internalType":"struct Miner.WithdrawArgs","name":"_args","type":"tuple"}],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes","name":"_proof","type":"bytes"},{"components":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"bytes32","name":"extDataHash","type":"bytes32"},{"components":[{"internalType":"uint256","name":"fee","type":"uint256"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"address","name":"relayer","type":"address"},{"internalType":"bytes","name":"encryptedAccount","type":"bytes"}],"internalType":"struct Miner.WithdrawExtData","name":"extData","type":"tuple"},{"components":[{"internalType":"bytes32","name":"inputRoot","type":"bytes32"},{"internalType":"bytes32","name":"inputNullifierHash","type":"bytes32"},{"internalType":"bytes32","name":"outputRoot","type":"bytes32"},{"internalType":"uint256","name":"outputPathIndices","type":"uint256"},{"internalType":"bytes32","name":"outputCommitment","type":"bytes32"}],"internalType":"struct Miner.AccountUpdate","name":"account","type":"tuple"}],"internalType":"struct Miner.WithdrawArgs","name":"_args","type":"tuple"},{"internalType":"bytes","name":"_treeUpdateProof","type":"bytes"},{"components":[{"internalType":"bytes32","name":"oldRoot","type":"bytes32"},{"internalType":"bytes32","name":"newRoot","type":"bytes32"},{"internalType":"bytes32","name":"leaf","type":"bytes32"},{"internalType":"uint256","name":"pathIndices","type":"uint256"}],"internalType":"struct Miner.TreeUpdateArgs","name":"_treeUpdateArgs","type":"tuple"}],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"withdrawVerifier","outputs":[{"internalType":"contract IVerifier","name":"","type":"address"}],"stateMutability":"view","type":"function"}]'

In [5]:
'''
Create the contract object to decode the input data of txs.
'''

w3 = Web3(Web3.HTTPProvider("https://cloudflare-eth.com"))
contract = w3.eth.contract(address=w3.toChecksumAddress(miner_address), abi=eval(miner_abi))


In the function params '_args' is the data that we are interested in.
The first argument is the amount of anonimity points being converted to TORN.
The third argument is a tuple, where the second and third elements of it are the
recipient address and relayer address, respectively. This can be checked when decoding
input data of such a tx in Etherscan. For example,
https://etherscan.io/tx/0xff238049835a3799159d93e130b6d8d4d981162dea42b522c81ec7c47a8178ac


In [6]:
'''
Decodes input field of a miner tx.

Returns a tuple of three elements:
 1. Function call
 2. Anonimity points
 3. Recipient address

When function call is not of type 'withdraw', 2. and 3. are NaN.
'''

def decode_miner_txs_input(tx, contract):
    func_object, func_params = contract.decode_function_input(tx.input)
    f = str(func_object)
    
    fn_call = "withdraw" if re.search("withdraw", f) else "reward" if re.search("reward", f) else "other"

    if fn_call in ["reward", "other"]:
        return (fn_call, np.nan, np.nan)
    else:
        anonimity_points = func_params["_args"][0]
        recipient_address = func_params["_args"][2][1]
        return (fn_call, anonimity_points, recipient_address.lower())

In [ ]:
decoded_info = miner_txs.progress_apply(lambda row: decode_miner_txs_input(row, contract), axis=1)

  4%|▍         | 1340/33645 [00:03<01:10, 457.69it/s]

In [ ]:
'''
Get three lists from the decoded_info results.

function calls:
    'reward' -> r
    'withdraw' -> w
'''

fn_calls = np.empty(len(decoded_info), dtype=str)
anonimity_points = np.empty(len(decoded_info), dtype=object)
recipient_addresses = np.empty(len(decoded_info), dtype=object)

for i in range(len(decoded_info)):
    fn_calls[i] = decoded_info[i][0]
    anonimity_points[i] = decoded_info[i][1]
    recipient_addresses[i] = decoded_info[i][2]

In [ ]:
'''
Set new columns to the miner_txs dataframe
'''
miner_txs["function_call"] = fn_calls
miner_txs["anonimity_points"] = anonimity_points
miner_txs["recipient_address"] = recipient_addresses

In [ ]:
'''
Drop input field so that data is lighter, now that we have extracted the 
necessary information
'''
no_input_miner_txs = miner_txs.drop(columns=["input"])

In [ ]:
# no_input_miner_txs.to_csv("../data/lighter_miner_txs.csv")